# Notebook Title

## Setup Python and R environment
you can ignore this section

In [2]:
%load_ext rpy2.ipython
%load_ext autoreload
%autoreload 2

%matplotlib inline  
from matplotlib import rcParams
rcParams['figure.figsize'] = (16, 100)

import warnings
from rpy2.rinterface import RRuntimeWarning
warnings.filterwarnings("ignore") # Ignore all warnings
# warnings.filterwarnings("ignore", category=RRuntimeWarning) # Show some warnings

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import display, HTML

During startup - Warning messages:
1: Setting LC_COLLATE failed, using "C" 
2: Setting LC_TIME failed, using "C" 
3: Setting LC_MESSAGES failed, using "C" 
4: Setting LC_MONETARY failed, using "C" 


In [3]:
%%javascript
// Disable auto-scrolling
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

<IPython.core.display.Javascript object>

In [4]:
%%R

# My commonly used R imports

require('tidyverse')

── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.4     ✔ readr     2.1.5
✔ forcats   1.0.0     ✔ stringr   1.5.1
✔ ggplot2   3.4.4     ✔ tibble    3.2.1
✔ lubridate 1.9.3     ✔ tidyr     1.3.0
✔ purrr     1.0.2     
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors


Loading required package: tidyverse


## Load & Clean Data

👉 Load the data along with the census connectors below (the output of the `connect-to-census.ipynb` notebook) and do any cleanup you'd like to do.

In [5]:
%%R

# load the data baltimore-lmop-with-geocodes.csv

df <- read.csv('baltimore-lmop-with-geocodes.csv')


## 👉 Grab Census Data

1. loading the Census API key

In [6]:
import dotenv

# Load the environment variables
# (loads CENSUS_API_KEY from .env)
dotenv.load_dotenv()


True

In [7]:
%%R 

require('tidycensus')

# because it an environment variable, we don't have to 
# explicitly pass this string to R, it is readable here
# in this R cell.
census_api_key(Sys.getenv("CENSUS_API_KEY"))



Loading required package: tidycensus
To install your API key for use in future sessions, run this function with `install = TRUE`.


2. Decide which Census variables you want

    Use <https://censusreporter.org/> to figure out which tables you want. (if censusreporter is down, check out the code in the cell below)

    -   Scroll to the bottom of the page to see the tables.
    -   If you already know the table ID, stick that in the "Explore" section to learn more about that table.

    By default this code loads (B01003_001) which we found in censusreporter here: https://censusreporter.org/tables/B01003/

    - find some other variables that you're also interested in
    - don't forget to pick a geography like "tract", "county" or "block group". here is the list of [all geographies](https://walker-data.com/tidycensus/articles/basic-usage.html#geography-in-tidycensus
    ).


In [8]:
%%R 

# Finding the Census Varaibles for the ACS 5 year survey
# Generall you'd do this in CensusReporter, but since it's down sometimes, here it is using tidycensus's load_variables function

# get every single variable in the ACS5
all_census_vars <- load_variables(2021, "acs5", cache = TRUE) 

filtered_census_vars <- all_census_vars %>% 
    filter(grepl("median income", label, ignore.case = TRUE))   # filter to those containing "median income"
    
# write to CSV so we can view it in python
filtered_census_vars %>% 
    write_csv("filtered_census_vars.csv")

# show the first few rows
filtered_census_vars %>%
    select(-geography) %>% # remove the geography column
    print(n = 20) # print the first 20 rows

# A tibble: 46 × 3
   name         label                                                    concept
   <chr>        <chr>                                                    <chr>  
 1 B06011PR_001 Estimate!!Median income in the past 12 months --!!Total: MEDIAN…
 2 B06011PR_002 Estimate!!Median income in the past 12 months --!!Total… MEDIAN…
 3 B06011PR_003 Estimate!!Median income in the past 12 months --!!Total… MEDIAN…
 4 B06011PR_004 Estimate!!Median income in the past 12 months --!!Total… MEDIAN…
 5 B06011PR_005 Estimate!!Median income in the past 12 months --!!Total… MEDIAN…
 6 B06011_001   Estimate!!Median income in the past 12 months --!!Total: MEDIAN…
 7 B06011_002   Estimate!!Median income in the past 12 months --!!Total… MEDIAN…
 8 B06011_003   Estimate!!Median income in the past 12 months --!!Total… MEDIAN…
 9 B06011_004   Estimate!!Median income in the past 12 months --!!Total… MEDIAN…
10 B06011_005   Estimate!!Median income in the past 12 months --!!Total… MEDIAN…
11 B07011

In [31]:
%%R 
# the variable B01003_001E was selectd from the census table 
# for population, which we found in censusreporter here:
# https://censusreporter.org/tables/B01003/

# in the table below, pick the geography, the variables, and the survey you want to pull from
# see the possible values here https://walker-data.com/tidycensus/articles/basic-usage.html

# Get variable from ACS, Baltimore, median income, and population in 2021
md_census_data <- get_acs(geography = "tract", 
                      state='MD',
                      variables = c(
                            pop="B01003_001E",
                            median_age="B01002_001E",
                            median_inc="B19013_001E"
                      ), 
                      year = 2021,
                      survey="acs5",
                      geometry=T)

md_census_data

Simple feature collection with 1475 features and 5 fields (with 10 geometries empty)
Geometry type: MULTIPOLYGON
Dimension:     XY
Bounding box:  xmin: -79.48765 ymin: 37.91172 xmax: -75.04894 ymax: 39.72304
Geodetic CRS:  NAD83
First 10 features:
         GEOID                                                   NAME
1  24033800403 Census Tract 8004.03, Prince George's County, Maryland
2  24510160300            Census Tract 1603, Baltimore city, Maryland
3  24033800217 Census Tract 8002.17, Prince George's County, Maryland
4  24019970900         Census Tract 9709, Dorchester County, Maryland
5  24510020100             Census Tract 201, Baltimore city, Maryland
6  24031703206      Census Tract 7032.06, Montgomery County, Maryland
7  24005402602       Census Tract 4026.02, Baltimore County, Maryland
8  24510190300            Census Tract 1903, Baltimore city, Maryland
9  24510271801         Census Tract 2718.01, Baltimore city, Maryland
10 24031701417      Census Tract 7014.17, Montgomery

Getting data from the 2017-2021 5-year ACS
Using FIPS code '24' for state 'MD'


In [10]:
%%R
# put census data into pd dataframe
md_census_df <- as.data.frame(md_census_data)


In [11]:
%%R
# md census data to csv
write_csv(md_census_df, "md_census_data.csv")

## 👉 Merge it with your data

hint...`tidycensus` provides you data in long format you may need to pivot the census data from long to wide format before merging it with your data

In [12]:
# merging data with python pandas

# read in the csv
md_census_df = pd.read_csv('md_census_data.csv')

# read in the csv
df = pd.read_csv('baltimore-lmop-with-geocodes.csv')

In [13]:
# merge the data on the GEOID
merged = pd.merge(df, md_census_df, left_on='GEOID', right_on='GEOID')

merged

,GHGRP ID,Landfill ID,Landfill Name,State,Physical Address,City,County,Zip Code,lat,long,...,GEOID,STATE,COUNTY,TRACT,BLOCK,NAME,variable,estimate,moe,geometry


In [14]:
# merging on TRACT instead of GEOID
#inspect df TRACT
df['TRACT']

0     603001
1     751600
2     861003
3     970200
4     260402
5     800607
6     800607
7     800607
8      30903
9     951200
10    850904
11    411302
12    411302
13    751201
14    740106
15     11700
16       601
17    751102
18    701211
19    701211
20    305200
21    411302
22    514201
23    960502
24    960502
25    955201
26    741000
27      1900
28    602700
29     10300
30     10300
31    452400
32    504100
33    700103
34    700103
35    970200
36    250500
37    850300
38    250500
39    751904
40     11700
41       601
42    876004
43    800411
44    800411
45    730102
46    708004
47    930800
Name: TRACT, dtype: int64

In [15]:
#inspect md_census_df head
md_census_df.head()

,GEOID,NAME,variable,estimate,moe,geometry
0,24033800403,"Census Tract 8004.03, Prince George's County, ...",B01002_001,38.8,4.6,"list(list(c(-76.761151, -76.761156, -76.760475..."
1,24033800403,"Census Tract 8004.03, Prince George's County, ...",B01003_001,3932.0,453.0,"list(list(c(-76.761151, -76.761156, -76.760475..."
2,24033800403,"Census Tract 8004.03, Prince George's County, ...",B19013_001,156014.0,21998.0,"list(list(c(-76.761151, -76.761156, -76.760475..."
3,24510160300,"Census Tract 1603, Baltimore city, Maryland",B01002_001,55.3,11.9,"list(list(c(-76.646118, -76.644643, -76.643218..."
4,24510160300,"Census Tract 1603, Baltimore city, Maryland",B01003_001,1578.0,791.0,"list(list(c(-76.646118, -76.644643, -76.643218..."


In [25]:
# create new column in md_census_df called TRACT that will only get Census Tract from NAME
md_census_df['TRACT'] = md_census_df['NAME'].str.extract(r'(\d+.\d+)')

# remove . from TRACT
md_census_df['TRACT'] = md_census_df['TRACT'].str.replace('.', '')
md_census_df.head()

# change TRACT to int make nan 0
md_census_df['TRACT'] = md_census_df['TRACT'].fillna(0).astype(int)


In [26]:
md_census_df['TRACT']

0       800403
1       800403
2       800403
3         1603
4         1603
         ...  
4420      9900
4421      9900
4422      9900
4423      9900
4424      9900
Name: TRACT, Length: 4425, dtype: int64

In [29]:
# save md_census_df to csv
md_census_df.to_csv('md_census_data.csv', index=False)

In [28]:
# merge the data on the TRACT
merged = pd.merge(df, md_census_df, left_on='TRACT', right_on='TRACT')

merged

,GHGRP ID,Landfill ID,Landfill Name,State,Physical Address,City,County,Zip Code,lat,long,...,STATE,COUNTY,TRACT,BLOCK,GEOID_y,NAME,variable,estimate,moe,geometry
0,1007291.0,734,Alpha Ridge SLF,MD,2350 Marriottsville Road,Marriottsville,Howard,21104.0,39.305776,-76.898803,...,24,27,603001,3008,24027603001,"Census Tract 6030.01, Howard County, Maryland",B01002_001,46.7,1.6,"list(list(c(-76.930591, -76.930146, -76.928361..."
1,1007291.0,734,Alpha Ridge SLF,MD,2350 Marriottsville Road,Marriottsville,Howard,21104.0,39.305776,-76.898803,...,24,27,603001,3008,24027603001,"Census Tract 6030.01, Howard County, Maryland",B01003_001,6212.0,502.0,"list(list(c(-76.930591, -76.930146, -76.928361..."
2,1007291.0,734,Alpha Ridge SLF,MD,2350 Marriottsville Road,Marriottsville,Howard,21104.0,39.305776,-76.898803,...,24,27,603001,3008,24027603001,"Census Tract 6030.01, Howard County, Maryland",B19013_001,125298.0,11749.0,"list(list(c(-76.930591, -76.930146, -76.928361..."
3,NaN,736,Appeal SLF,MD,NaN,Lusby,Calvert,20657.0,38.381112,-76.438334,...,24,9,861003,3000,24009861003,"Census Tract 8610.03, Calvert County, Maryland",B01002_001,38.9,6.5,"list(list(c(-76.452533, -76.448208, -76.444544..."
4,NaN,736,Appeal SLF,MD,NaN,Lusby,Calvert,20657.0,38.381112,-76.438334,...,24,9,861003,3000,24009861003,"Census Tract 8610.03, Calvert County, Maryland",B01003_001,6548.0,914.0,"list(list(c(-76.452533, -76.448208, -76.444544..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
97,1004563.0,20860,Solley Road Landfill,MD,7890 Solley Road,Glen Burnie,Anne Arundel,21061.0,39.160313,-76.575042,...,24,3,730102,3040,24003730102,"Census Tract 7301.02, Anne Arundel County, Mar...",B01003_001,11304.0,1202.0,"list(list(c(-76.586788, -76.584974, -76.583188..."
98,1004563.0,20860,Solley Road Landfill,MD,7890 Solley Road,Glen Burnie,Anne Arundel,21061.0,39.160313,-76.575042,...,24,3,730102,3040,24003730102,"Census Tract 7301.02, Anne Arundel County, Mar...",B19013_001,108541.0,4097.0,"list(list(c(-76.586788, -76.584974, -76.583188..."
99,NaN,770,Sudley Road SLF,MD,5400 Nutwell Sudley Road,Deale,Anne Arundel,20751.0,38.817900,-76.578500,...,24,3,708004,4000,24003708004,"Census Tract 7080.04, Anne Arundel County, Mar...",B01002_001,35.7,6.1,"list(list(c(-76.71439, -76.713214, -76.712251,..."
100,NaN,770,Sudley Road SLF,MD,5400 Nutwell Sudley Road,Deale,Anne Arundel,20751.0,38.817900,-76.578500,...,24,3,708004,4000,24003708004,"Census Tract 7080.04, Anne Arundel County, Mar...",B01003_001,6421.0,1047.0,"list(list(c(-76.71439, -76.713214, -76.712251,..."


In [30]:
# save merged to csv
merged.to_csv('merged.csv', index=False)